# 📓 MLflow - Ejemplos básicos
# 
En esta notebook veremos ejemplos sencillos de:
- MLflow Tracking
- MLflow Projects
- MLflow Models
- MLflow Model Registry

## Instalación y configuración inicial
Instalación de MLflow (descomentar si no lo tienes instalado)
!pip install mlflow


In [1]:
# ========================
# MLflow - Ejemplos básicos
# ========================

# Instalación (si es necesario)
# !pip install mlflow

import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import os
import yaml
import pandas as pd
# Crear carpeta para artefactos si no existe
os.makedirs("outputs", exist_ok=True)


## 🧩 1. MLflow Tracking
 
### 📄 Descripción
Con **MLflow Tracking** registramos:
 - Parámetros (por ejemplo, hiperparámetros de un modelo).
 - Métricas (precisión, error, R2, etc.).
 - Artefactos (modelos entrenados, imágenes, datasets).
 
Todo queda guardado en un historial organizado para análisis y comparación.


## Nombre del proyecto y url de MLFlow local

In [2]:
#mlflow.set_tracking_uri("http://localhost:5000")  # o la IP donde esté el Docker
mlflow.set_experiment("Regresion lineal 4")

2025/05/11 21:37:29 INFO mlflow.tracking.fluent: Experiment with name 'Regresion lineal 4' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/guill/OneDrive/Documentos/simplegit/ITBA/mlruns/920470422329662775', creation_time=1747010249879, experiment_id='920470422329662775', last_update_time=1747010249879, lifecycle_stage='active', name='Regresion lineal 4', tags={}>

In [3]:
# Datos ficticios
X, y = make_regression(n_samples=100, n_features=1, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 2. Guardar los datasets como CSV
os.makedirs("data", exist_ok=True)
pd.DataFrame(X_train, columns=["x"]).to_csv("data/X_train.csv", index=False)
pd.DataFrame(X_test, columns=["x"]).to_csv("data/X_test.csv", index=False)
pd.DataFrame(y_train, columns=["y"]).to_csv("data/y_train.csv", index=False)
pd.DataFrame(y_test, columns=["y"]).to_csv("data/y_test.csv", index=False)


In [3]:
# Ejemplo sencillo de Tracking


# Empezar una corrida (run)
with mlflow.start_run(run_name="linear_regression_example 2"):
    # Modelo
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Parámetro (en este caso no hay hiperparámetros, así que lo simulamos)
    mlflow.log_param("model_type", "LinearRegression")
    
    # Métrica
    score = model.score(X_test, y_test)
    mlflow.log_metric("r2_score", score)
    
    # Guardar el modelo
    mlflow.sklearn.log_model(model, artifact_path="model")
    
    print(f"Modelo guardado con R2: {score:.2f}")


2025/05/11 20:54:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado con R2: 1.00


In [4]:
from sklearn.linear_model import Ridge

with mlflow.start_run(run_name="ridge_regression_example 3"):
    alpha = 0.7
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)

    mlflow.log_param("model_type", "Ridge")
    mlflow.log_param("alpha", alpha)

    score = model.score(X_test, y_test)
    mlflow.log_metric("r2_score", score)

    mlflow.sklearn.log_model(model, artifact_path="model")
    print(f"Modelo Ridge guardado con R2: {score:.2f}")
    X_train_path = "data/X_train.csv"
    X_test_path = "data/X_test.csv"
    y_train_path = "data/y_train.csv"
    y_test_path = "data/y_test.csv"
    mlflow.log_artifact(X_train_path, artifact_path="datasets")
    mlflow.log_artifact(X_test_path, artifact_path="datasets")
    mlflow.log_artifact(y_train_path, artifact_path="datasets")
    mlflow.log_artifact(y_test_path, artifact_path="datasets")
    

2025/05/11 21:37:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo Ridge guardado con R2: 1.00


In [5]:
import mlflow
import numpy as np

with mlflow.start_run(run_name="training_with_curve"):
    for epoch in range(10):
        # Simulación de loss que baja
        loss = np.exp(-epoch / 5)

        # Logueás la métrica con `step` para que MLflow genere la curva
        mlflow.log_metric("loss", loss, step=epoch)


In [53]:
mlflow.set_experiment("Regresion lineal 6")

mlflow.set_tag("model_name", "RandomForest")
mlflow.set_tag("experiment", "baseline")

models = {"LinearRegression": LinearRegression(), "Ridge": Ridge(alpha=0.5)}



In [56]:
mlflow.end_run()
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        mlflow.set_tag("model_name", name)

        for epoch in range(10):
            loss = np.exp(-epoch / 5)  # Ejemplo de pérdida
            mlflow.log_metric("loss", loss, step=epoch)

        mlflow.sklearn.log_model(model, artifact_path="model")



2025/05/11 22:42:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/11 22:42:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [58]:
mlflow.set_experiment("Seleccionar mejor modelo")

import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Datos ficticios
X, y = make_regression(n_samples=100, n_features=1, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Modelos a evaluar
models = {"LinearRegression": LinearRegression(), "Ridge": Ridge(alpha=0.5)}

best_model = None
best_score = -np.inf  # Mejor score encontrado (por ejemplo, R2 o MSE)

# Comenzamos los experimentos para cada modelo
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        mlflow.set_tag("model_name", name)

        # Entrenamiento
        model.fit(X_train, y_train)

        # Predicciones y métricas
        y_pred = model.predict(X_test)
        score = mean_squared_error(y_test, y_pred)  # Usamos MSE como ejemplo
        
        # Registro de parámetros y métricas
        mlflow.log_param("model_type", name)
        mlflow.log_metric("mse", score)
        
        # Guardamos el modelo
        mlflow.sklearn.log_model(model, artifact_path="model")
        
        # Evaluamos cuál es el mejor modelo
        if score < best_score:  # MSE más bajo es mejor
            best_score = score
            best_model = model
            mlflow.log_param("best_model", True)  # Marcamos el modelo ganador

# Ahora puedes registrar el mejor modelo global
if best_model:
    with mlflow.start_run(run_name="Best_Model"):
        mlflow.log_param("best_model_name", best_model.__class__.__name__)
        mlflow.sklearn.log_model(best_model, artifact_path="best_model")
        print("Modelo ganador registrado!")


2025/05/11 22:50:38 INFO mlflow.tracking.fluent: Experiment with name 'Seleccionar mejor modelo' does not exist. Creating a new experiment.
2025/05/11 22:50:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/11 22:50:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## 🧩 2. MLflow Projects
 
### 📄 Descripción
**MLflow Projects** define un estándar para empaquetar proyectos de ML, haciéndolos:
 - Reproducibles (cualquiera puede correrlo igual).
 - Ejecutables localmente o en la nube.
 - Versionables junto al código.
 
<!-- Utiliza un archivo `MLproject` (YAML) para describir dependencias y comandos de entrada. -->


In [8]:
# Creamos un archivo MLproject para definir el proyecto

project_yaml = """
name: simple_linear_regression
conda_env: conda.yaml

entry_points:
  main:
    parameters:
      alpha: {type: float, default: 0.5}
    command: "python train.py --alpha {alpha}"
"""

with open("MLproject", "w") as f:
    f.write(project_yaml)

# Creamos un entorno conda de ejemplo
conda_yaml = """
name: simple-mlflow-env
dependencies:
  - python=3.8
  - scikit-learn
  - pip
  - pip:
      - mlflow
"""

with open("conda.yaml", "w") as f:
    f.write(conda_yaml)

print("Archivo MLproject y conda.yaml creados 🎯")


Archivo MLproject y conda.yaml creados 🎯


## 🧩 3. MLflow Models
 
### 📄 Descripción
**MLflow Models** permite:
 - Guardar modelos entrenados en formatos estándar.
 - Cargarlos fácilmente para predicción o despliegue.
 - Exportarlos a múltiples plataformas (Docker, REST API, mobile).

In [9]:
# Guardamos el modelo entrenado en disco
model_path = "outputs/linear_model"
mlflow.sklearn.save_model(model, model_path)

# Cargamos el modelo desde disco
loaded_model = mlflow.sklearn.load_model(model_path)

# Realizamos una predicción de prueba
prediction = loaded_model.predict(X_test[:2])
print("Predicción ejemplo:", prediction)


Predicción ejemplo: [-55.4786619   61.72358202]


## 🧩 4. MLflow Model Registry
 
### 📄 Descripción
**MLflow Model Registry** gestiona:
 - Versiones de modelos.
 - Etapas del ciclo de vida (Staging, Production, Archived).
 - Aprobaciones y revisiones de modelos.
 
 **Importante:** Para usarlo realmente se necesita un Tracking Server conectado a una base de datos.
 Aquí simulamos un ejemplo sencillo en local.

In [10]:
# Buscar si el experimento ya existe
experiment_name = "Model_Registry_Example"

experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"Experimento '{experiment_name}' creado 🎯")
else:
    experiment_id = experiment.experiment_id
    print(f"Experimento '{experiment_name}' ya existía, usando id {experiment_id} ✔️")

# Seteamos el experimento como activo
mlflow.set_experiment(experiment_name)

# Registramos el modelo
with mlflow.start_run(run_name="registry_test_run"):
    mlflow.sklearn.log_model(model, "model", registered_model_name="LinearRegressionModel")



Experimento 'Model_Registry_Example' ya existía, usando id 3 ✔️


2025/05/11 19:43:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'LinearRegressionModel' already exists. Creating a new version of this model...
2025/05/11 19:43:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LinearRegressionModel, version 5
Created version '5' of model 'LinearRegressionModel'.


🏃 View run registry_test_run at: http://localhost:5000/#/experiments/3/runs/c11e00b1a53841a2bee9de21e3c6933f
🧪 View experiment at: http://localhost:5000/#/experiments/3
